In [6]:
from load import all_subreddits_data, tv_subreddits_data
import numpy as np
from sklearn.cross_validation import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier



In [11]:
import numpy as np
import marisa_trie
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import six

class MarisaCountVectorizer(CountVectorizer):

    # ``CountVectorizer.fit`` method calls ``fit_transform`` so
    # ``fit`` is not provided
    def fit_transform(self, raw_documents, y=None):
        X = super(MarisaCountVectorizer, self).fit_transform(raw_documents)
        X = self._freeze_vocabulary(X)
        return X

    def _freeze_vocabulary(self, X=None):
        if not self.fixed_vocabulary_:
            frozen = marisa_trie.Trie(six.iterkeys(self.vocabulary_))
            if X is not None:
                X = self._reorder_features(X, self.vocabulary_, frozen)
            self.vocabulary_ = frozen
            self.fixed_vocabulary_ = True
            del self.stop_words_
        return X

    def _reorder_features(self, X, old_vocabulary, new_vocabulary):
        map_index = np.empty(len(old_vocabulary), dtype=np.int32)
        for term, new_val in six.iteritems(new_vocabulary):
            map_index[new_val] = old_vocabulary[term]
        return X[:, map_index]

In [12]:
all_data = all_subreddits_data()


In [13]:

document_label = []
for label, docs in all_data.items():
    for d in docs:
        document_label.append((label, d))
label_index = { label: i for i, label in enumerate(all_data.keys()) }
index_label  = { i: label for i, label in enumerate(all_data.keys()) }

In [14]:
vectorizer = MarisaCountVectorizer(min_df=2, stop_words='english')
X = vectorizer.fit_transform(map(lambda x: x[1], document_label))
Y = np.array(map(lambda x: label_index[x[0]], document_label))


In [15]:
classifier = DecisionTreeClassifier()
scores = (cross_val_score(classifier, X.toarray(), Y, cv=10))

In [16]:
#anime vs. buildapc
hlda = 0.9961538
lda = 0.87692
dt = 0.97452149568428637
np.mean(scores)

0.6738541466574024

In [ ]:
#10 subreddits
hlda = 1386./1706